In [ ]:
import pandas

df = pandas.read_csv('./out.csv', quotechar='\'')

In [ ]:
import re
import string
import demoji
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from sklearn.feature_extraction.text import CountVectorizer

demoji.download_codes()
def clean_tweets(data):
    new_data = data.copy()

    username_hash = r'[#@]\w+'
    punctuation = '[%s]+' % re.escape(string.punctuation)
    special_char = r'[^0-9a-zA-Z\s]+'
    number = r'[0-9]+'
    space = r'\s{2,}'
    space_begin_end = r'^\s+|\s+$'
    url = r'(https?|www):\/{1,}\w+\W+\w+\/{1,}\w+'
    char_ref = r'&\w+;'

    for i in range(len(new_data)):
        new_data[i] = re.sub(char_ref, ' ', new_data[i])
        new_data[i] = re.sub(username_hash, '', new_data[i])
        new_data[i] = re.sub(url, '', new_data[i])
        new_data[i] = re.sub(punctuation, '', new_data[i])
        new_data[i] = re.sub(number, '', new_data[i])
        new_data[i] = re.sub(space_begin_end, '', new_data[i])
        new_data[i] = re.sub(space, '', new_data[i])
        new_data[i] = demoji.replace(new_data[i], '')
        new_data[i] = re.sub(special_char, '', new_data[i])

    return new_data

def case_fold(data):
    new_data = data.copy()
    return list(map(lambda s: s.lower(), new_data))

def tokenize(data):
    new_data = data.copy()
    return list(map(lambda s: s.split(' '), new_data))

def stem(data):
    new_data = data.copy()
    stemmer = StemmerFactory().create_stemmer()

    return list(map(lambda s: stemmer.stem(s), new_data))

def get_term(data):
    new_data = data.copy()
    count_vectorizer = CountVectorizer()
    count_vectorizer.fit(new_data)

    return count_vectorizer.get_feature_names()

In [ ]:
labels = df['category']
tweets = stem(case_fold(clean_tweets(df['tweet'])))
term = get_term(tweets)

print(term)

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder

# X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.2, random_state=13)

# enc = LabelEncoder()
# y_train = enc.fit_transform(y_train)
# y_test = enc.fit_transform(y_test)

# print(y_train)
# print(y_test)

In [ ]:
# from sklearn.feature_extraction.text import TfidfVectorizer

# vect = TfidfVectorizer()
# vect.fit(tweets)

# X_train_tfidf = vect.transform(X_train)
# X_test_tfidf = vect.transform(X_test)

# print(vect.vocabulary_)

In [ ]:
# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score

# svm_c = SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
# svm_c.fit(X_train_tfidf, y_train)
# pred = svm_c.predict(X_test_tfidf)

# print(accuracy_score(pred, y_test))

In [ ]:
# from sklearn.naive_bayes import MultinomialNB

# nb = MultinomialNB()
# nb.fit(X_train_tfidf, y_train)
# pred = nb.predict(X_test_tfidf)

# print(accuracy_score(pred, y_test))